In [1]:
from entsoe import load_data
import numpy as np
from torch_geometric_temporal.nn.attention.stgcn import STConv
import torch
import torch.nn as nn
from tqdm import tqdm

In [2]:
data = load_data()

torch.Size([43729, 10, 24]) torch.Size([43729, 2, 32]) torch.Size([43729, 32, 1]) torch.Size([43729, 32, 1])


In [ ]:
class FullyConnLayer(nn.Module):
    def __init__(self, c):
        super(FullyConnLayer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        return self.conv(x)


# adapted from Hao Wei
class OutputLayer(nn.Module):
    def __init__(self, c, T, n):
        super(OutputLayer, self).__init__()
        self.tconv1 = nn.Conv2d(c, c, (T, 1), 1, dilation=1, padding=(0, 0))
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = nn.Conv2d(c, c, (1, 1), 1, dilation=1, padding=(0, 0))
        self.fc = FullyConnLayer(c)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)


class Model(nn.Module):
    def __init__(self, window_size, num_nodes):
        super(Model, self).__init__()
        self.stconv1 = STConv(
            num_nodes=num_nodes,
            in_channels=24,
            hidden_channels=16,
            out_channels=64,
            kernel_size=3,
            K=1,
        )
        self.stconv2 = STConv(
            num_nodes=num_nodes,
            in_channels=64,
            hidden_channels=16,
            out_channels=64,
            kernel_size=3,
            K=1,
        )
        # window_size - 2 * num_layers * (kernel_size - 1) = 24 - 2 * 2 * (3 - 1) = 16
        T = window_size - 2 * 2 * (3 - 1)
        self.output_layer = OutputLayer(64, T, 10)

    def forward(self, x, edge_index, edge_weight):
        x = self.stconv1(x, edge_index, edge_weight)
        x = self.stconv2(x, edge_index, edge_weight)
        x = x.permute(0, 3, 1, 2)
        x = self.output_layer(x)
        x = x.permute(0, 2, 3, 1)
        return x

In [12]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, x, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [14]:
# len_data = len(data)
len_data = 730
window_size = 24
future = 1
model = Model(window_size, 10)
model.train()
edge_index = data[0].edge_index  # static graph
# print(edge_index.shape)
criteron_mse = nn.MSELoss()
criteron_mae = nn.L1Loss()
criteron_rmse = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(100):
    epoch_loss_mse = 0
    epoch_loss_mae = 0
    epoch_loss_rmse = 0
    for i in tqdm(range(0, len_data - window_size - future)):
        optimizer.zero_grad()
        window_data = data[i : i + window_size]
        X = [d.x for d in window_data]
        X = torch.stack(X, dim=0)
        # Add a 1 dimension in front as batch dimension
        X = X.unsqueeze(0)
        # print(X.shape)
        edge_weights = [d.edge_attr for d in window_data]
        avg_edge_weight = np.mean(edge_weights, axis=0)
        edge_weights = torch.tensor(avg_edge_weight, dtype=torch.float32)

        y = data[i + window_size + future - 1].x[:, 0]  # first feature of a node is DAP
        # print(y.shape)
        y_hat = model(X, edge_index, edge_weights)
        # print(y_hat)
        # print(y_hat.shape)
        y_hat = y_hat.squeeze(0, 1, 3)
        # print(y_hat.shape)
        loss_mse = criteron_mse(y_hat, y)
        loss_mae = criteron_mae(y_hat, y)
        loss_rmse = criteron_rmse(y_hat, y)

        epoch_loss_mse += loss_mse.item()
        epoch_loss_mae += loss_mae.item()
        epoch_loss_rmse += loss_rmse.item()
        loss_mse.backward()
        optimizer.step()
    print(f"Epoch {epoch}")
    print(epoch_loss_mse / (len_data - window_size - 1))
    print(epoch_loss_mae / (len_data - window_size - 1))
    print(epoch_loss_rmse / (len_data - window_size - 1))

100%|██████████| 705/705 [00:11<00:00, 62.59it/s]


Epoch 0
0.4651564080413457
0.41686361099177216
0.5418906099724431


100%|██████████| 705/705 [00:11<00:00, 63.92it/s]


Epoch 1
0.328510635304562
0.3586041071524857
0.4662670336610882


100%|██████████| 705/705 [00:10<00:00, 64.22it/s]


...

Epoch 97
0.06606375700232725
0.1831419752527636
0.23435598268893593


100%|██████████| 705/705 [00:10<00:00, 64.86it/s]


Epoch 98
0.06315289997290952
0.17947952106383674
0.22846645895682327


100%|██████████| 705/705 [00:10<00:00, 65.45it/s]

Epoch 99
0.06284585300674464
0.179225962096495
0.22769988079865774
